In [1]:
import requests
import yaml

### Testing the LibCal API's

In [2]:
config_path = 'config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
client_id = config['LibCal']['client_id']
client_secret = config['LibCal']['client_secret']

In [4]:
credentials_endpt = 'https://booking.library.gwu.edu/1.1/oauth/token'
calendars_endpt = 'https://booking.library.gwu.edu/1.1/calendars'
bookings_endpt = 'https://booking.library.gwu.edu/1.1/space/bookings'

In [5]:
def create_cred_body(client_id: str, client_secret: str):
    '''Creates the body of a POST request for obtaining an access token, given a client secret and credentials.'''
    return {'client_id': client_id,
           'client_secret': client_secret,
           'grant_type': 'client_credentials'}

In [6]:
def get_access_token(cred_body: dict):
    '''Retrieves an access token from the LibCal API, given a request body as specified.'''
    try:
        resp = requests.post(credentials_endpt, json=cred_body)
        resp.raise_for_status()
        token_data = resp.json()
        if 'error' in token_data:
            raise Exception(token_data)
            #return resp
        return token_data
    except Exception as e:
        print(e)
        raise

In [7]:
cred_body = create_cred_body(client_id, client_secret)
token_data = get_access_token(cred_body)

In [8]:
def create_header(token_data: dict):
    '''Creates an authorization header with the supplied token from the credentials API.'''
    try:
        token = token_data['access_token']
        return {'Authorization': f'Bearer {token}'}
    except KeyError:
        print('Access token missing from token data.')
        raise

In [9]:
def get_calendars(header: dict):
    '''Retrieves the calendars available in the system, for querying events from a specific calendar.
    Header argument should contain the authorization token.'''
    try:
        resp = requests.get(calendars_endpt, headers=header)
        resp.raise_for_status()
        cal_data = resp.json()
        if 'error' in cal_data:
            raise Exception(cal_data)
        return cal_data
    except Exception as e:
        print('Error getting calendars.')
        raise
    

In [10]:
header = create_header(token_data)
calendars = get_calendars(header)

In [11]:
def create_booking_params(date: str = None):
    '''Sets querystr parameters for the space/bookings endpoint. 
    Date should be in the format YYYY-MM-DD.'''
    params = {'formAnswers': 1,
             'limit': 100} # Max value
    if date:
        params['date'] = date
    return params
        

In [12]:
def get_bookings(header: dict, booking_id: str = None, date: str = None):
    '''Retrieves bookings from LibCal.'''
    if booking_id:
        url = f'{bookings_endpt}/{booking_id}'
    else:
        url = bookings_endpt
    try:
        params = create_booking_params(date)
        resp = requests.get(url, headers=header, params=params)
        resp.raise_for_status()
        bookings_data = resp.json()
        if 'error' in bookings_data:
            raise Exception(bookings_data)
        return bookings_data
    except Exception as e:
        print('Error getting bookings')
        raise

I think the `space/bookings` endpoint is the one to use for the room reservation system. Note that it seems possible to retrieve appointments only for one day at a time. By default, the date is today's date. Another date can be supplied as a querystring parameter.

In [13]:
bookings = get_bookings(header)

We need to include a specific location in our query. This endpoint should return a list of locations from LibCal.

In [16]:
locations_endpt = 'https://booking.library.gwu.edu/1.1/space/locations'

In [17]:
locations = requests.get(locations_endpt, headers=header)

In [18]:
locations.json()

[{'lid': 8827,
  'name': 'Gelman Library Study Spaces',
  'public': 1,
  'formid': 4036},
 {'lid': 10138, 'name': 'Gelman Item Pick Up', 'public': 0},
 {'lid': 10332,
  'name': 'Virginia Science & Technology Campus Library',
  'public': 1},
 {'lid': 8836, 'name': 'Student Full-Semester Loaner Equipment', 'public': 1},
 {'lid': 8874, 'name': 'CREATE Digital Studio', 'public': 1}]

### Testing the Alma API's
Goal is to retrieve a user's barcode, given email from LibCal

In [19]:
def query_users(user_email: str):
    '''Queries the Alma Users API, given a user\'s  email address.'''
    params = {'q': f'email~{user_email}'}
    apikey = config['Alma']['apikey']
    headers = {'Authorization': f"apikey {apikey}",
              'Accept': 'application/json'}
    try:
        resp = requests.get(config['Alma']['users_endpt'], headers=headers, params=params)
        resp.raise_for_status() 
        user = resp.json()
        return user['user'][0]['link']
    except Exception as e:
        print(f'Error fetching user for email {user_email}')
        raise

In [20]:
def fetch_user(link: str):
    '''Given a link to a specific user via Alma\'s API, get the full user record in order to see the barcode.'''
    apikey = config['Alma']['apikey']
    headers = {'Authorization': f"apikey {apikey}",
              'Accept': 'application/json'}
    try:
        resp = requests.get(link, headers=headers)
        resp.raise_for_status() 
        user = resp.json()
        return user
    except Exception as e:
        print(f'Error fetching user for email {user_email}')
        return resp
        #raise

In [22]:
link = query_users(email)
user = fetch_user(link)

In [26]:
# New field for GWID
gwids = [b['q12505'] for b in bookings if 'q12505' in b]

### Testing SQLite integration

In [54]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
%aimport sqlite_cache

In [106]:
sqlc = sqlite_cache.SQLiteCache()

Tables already exist. Skipping table creation.


In [92]:
# Add dummy visitor ID for testing
for i, _ in enumerate(users):
    users[i].update({'visitor_id': i})
    users[i]

In [101]:
sqlc.add_users(users)

In [113]:
sqlc.user_lookup('G15791109')

In [108]:
sqlc.user_lookup('G15791109')

In [110]:
for i, b in enumerate(bookings):
    sqlc.add_appt({'appt_id': b['bookId'],
                  'prereg_id': i})

In [112]:
sqlc.appt_lookup('cs_JnMn5iv')